In [1]:
from scipy.linalg import expm
import numpy as np                                          # 导入numpy库并简写为np
import pandas as pd
import itertools
from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt
from QAOA import _Rx,_Rz,_Rzz,_Rzzz,_Rzzzz,I,_H,getBaseProb
def I(n): #返回可占用n个比特位的I矩阵
    return np.eye(2**n)

In [2]:
def QaoaGetPara(gamma,beta,t = np.array([1,0,0,0]).reshape(-1,1),isOutStat = False):
    z = np.array([[1,0],
                    [0,-1]],dtype=np.complex128)
    A = _Rx(0,2,beta)*_Rx(1,2,beta)*_Rz(0,2,gamma)*(_Rz(1,2,gamma))*(_Rzz(0,1,2,gamma))*(_H() + _H()) # 代表量子线路的矩阵
    B = A.dot(t) # |gamma,beta>
    print(B.shape)
    if isOutStat:
        return B
    H_C = np.kron(z,z) + np.kron(np.eye(2),z) + np.kron(z,np.eye(2)) # Hamiltonian
    E = np.conjugate(np.array(B)).T.dot(H_C).dot(np.array(B))[0][0].real # 计算期望
    return (E,gamma,beta)
t = np.array([1,0,0,0]).reshape(-1,1) # 初态
beta_lst = np.linspace(0,2*np.pi,200)
gamma_lst = np.linspace(0,2*np.pi,200) 
#*********** 并行版本 ******************
table = Parallel(n_jobs=-1)(delayed(QaoaGetPara)(beta,gamma,t) for beta,gamma in itertools.product(beta_lst,gamma_lst)) #并行版本
table = pd.DataFrame(table,columns = ["E","gamma","beta"])
row_idx = table["E"].idxmax()
row = table.loc[row_idx,["gamma", "beta"]] # E_max对应列
pureState = QaoaGetPara(row["gamma"],row["beta"],t,True) # 返回该最优量子线路输出的纯态
getBaseProb(pureState) # 返回每个基态的概率

#*********** 串行版本 ******************
"""
E_max = 0
para_dict = {"gamma":0,"beta":0}
beta_lst = np.linspace(0,2*np.pi,200)
gamma_lst = np.linspace(0,2*np.pi,200)
pbar = tqdm(total=len(beta_lst)*len(gamma_lst))
t = np.array([1,0,0,0]).reshape(-1,1) # 初态
for i,beta in enumerate(beta_lst): # 串行版本
    for j,gamma in enumerate(gamma_lst):
        #A = _Rx(0,2,beta)*_Rx(1,2,beta)*_Rz(0,2,gamma)*(_Rz(1,2,gamma))*(_Rzz(0,1,2,gamma))*(_H() + _H()) # 代表量子线路的矩阵
       # B = A.dot(t) # |gamma,beta>
       # H_C = np.kron(z,z) + np.kron(np.eye(2),z) + np.kron(z,np.eye(2)) # Hamiltonian
       # E = np.conjugate(np.array(B)).T.dot(H_C).dot(np.array(B))[0][0].real # 计算期望
        E = QaoaGetPara(gamma,beta)[0]
        if E > E_max:
            E_max = E
            para_dict["gamma"] = gamma
            para_dict["beta"] = beta
        else:
            pass
        pbar.update(1)

"""

(4, 1)


'\nE_max = 0\npara_dict = {"gamma":0,"beta":0}\nbeta_lst = np.linspace(0,2*np.pi,200)\ngamma_lst = np.linspace(0,2*np.pi,200)\npbar = tqdm(total=len(beta_lst)*len(gamma_lst))\nt = np.array([1,0,0,0]).reshape(-1,1) # 初态\nfor i,beta in enumerate(beta_lst): # 串行版本\n    for j,gamma in enumerate(gamma_lst):\n        #A = _Rx(0,2,beta)*_Rx(1,2,beta)*_Rz(0,2,gamma)*(_Rz(1,2,gamma))*(_Rzz(0,1,2,gamma))*(_H() + _H()) # 代表量子线路的矩阵\n       # B = A.dot(t) # |gamma,beta>\n       # H_C = np.kron(z,z) + np.kron(np.eye(2),z) + np.kron(z,np.eye(2)) # Hamiltonian\n       # E = np.conjugate(np.array(B)).T.dot(H_C).dot(np.array(B))[0][0].real # 计算期望\n        E = QaoaGetPara(gamma,beta)[0]\n        if E > E_max:\n            E_max = E\n            para_dict["gamma"] = gamma\n            para_dict["beta"] = beta\n        else:\n            pass\n        pbar.update(1)\n\n'

In [3]:
table

,E,gamma,beta
0,0.000000e+00,0.000000,0.000000
1,-1.084202e-19,0.000000,0.031574
2,5.551115e-17,0.000000,0.063148
3,5.724587e-17,0.000000,0.094721
4,-5.204170e-18,0.000000,0.126295
...,...,...,...
39995,-5.030698e-16,6.283185,6.156890
39996,5.811324e-17,6.283185,6.188464
39997,5.594483e-17,6.283185,6.220038
39998,3.252607e-19,6.283185,6.251612


In [11]:
getBaseProb(pureState) # 返回每个基态的概率
psi = pureState
rho = np.outer(psi, psi.conj())

In [14]:
getBaseProb(pureState) # 返回每个基态的概率


array([0.69440573+0.j, 0.02661554+0.j, 0.02661554+0.j, 0.2523632 +0.j])

In [4]:
# ------------------p = 1 template-----------------------------
def QaoaGetPara(gamma,beta,t,isOutStat = False):
    z = np.array([[1,0],
                    [0,-1]],dtype=np.complex128)
    A = _Rx(0,4,beta)*_Rx(1,4,beta)*_Rx(2,4,beta)*_Rx(3,4,beta) \
        *_Rz(0,4,gamma)*_Rz(1,4,gamma)*_Rz(2,4,gamma)*_Rz(3,4,gamma) \
        *_Rzz(0,1,4,gamma)*_Rzz(2,3,4,gamma) \
        *(_H() + _H() + _H() + _H()) #代表量子线路的矩阵
    B = A.dot(t) # |gamma,beta>
    if isOutStat:
        return B
    H_C = np.kron(z,I(3)) + np.kron(np.kron(I(1),z),I(2)) + np.kron(np.kron(I(2),z),I(1)) +  \
          np.kron(I(3),z) + np.kron(np.kron(z,z),I(2)) + np.kron(I(2),np.kron(z,z))
    E = np.conjugate(np.array(B)).T.dot(H_C).dot(np.array(B))[0][0].real # 计算期望
    return (E,gamma,beta)

t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1)
beta_lst = np.linspace(0,2*np.pi,100)
gamma_lst = np.linspace(0,2*np.pi,100) 
#*********** 并行版本 ******************
table = Parallel(n_jobs=-1)(delayed(QaoaGetPara)(beta,gamma,t) for beta,gamma in itertools.product(beta_lst,gamma_lst)) #并行版本
table = pd.DataFrame(table,columns = ["E","gamma","beta"])
row_idx = table["E"].idxmax()
row = table.loc[row_idx,["gamma", "beta"]] # E_max对应列
pureState = QaoaGetPara(row["gamma"],row["beta"],t,True) # 返回该最优量子线路输出的纯态
State = getBaseProb(pureState) # 返回每个基态的概率

In [11]:
State

array([0.48192761+0.j, 0.01838446+0.j, 0.01838446+0.j, 0.17551353+0.j,
       0.01838446+0.j, 0.00070133+0.j, 0.00070133+0.j, 0.00669545+0.j,
       0.01838446+0.j, 0.00070133+0.j, 0.00070133+0.j, 0.00669545+0.j,
       0.17551353+0.j, 0.00669545+0.j, 0.00669545+0.j, 0.06392039+0.j])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 创建示例数据框

# 将数据框转换为矩阵形式
matrix = table.round(4).pivot(index='gamma', columns='beta', values='E')

# 绘制热力图
sns.heatmap(matrix, cmap='coolwarm', fmt='.0f')

# 显示图形
plt.show()

In [ ]:
Xlabel = ["|{}{}{}{}⟩".format(x,y,z,r) for x,y,z,r in itertools.product([0,1],[0,1],[0,1],[0,1])]
plt.figure(figsize=(10,6))
plt.bar(Xlabel,State.real)

plt.show()

In [2]:
def QaoaGetPara(gamma,beta,t,isOutStat = False):# 这是p = 1 的 2th
    z = np.array([[1,0],
                    [0,-1]],dtype=np.complex128)
    A = _Rx(0,4,beta)*_Rx(1,4,beta)*_Rx(2,4,beta)*_Rx(3,4,beta) \
        *_Rz(0,4,gamma)*_Rz(1,4,gamma)*_Rz(2,4,gamma)*_Rz(3,4,gamma) \
        *(_Rzz(0,1,4,-gamma))*_Rzz(2,3,4,gamma)*_Rzz(0,2,4,gamma) \
        *(_H() + _H() + _H() + _H()) #代表量子线路的矩阵
    B = A.dot(t) # |gamma,beta>
    if isOutStat:
        return B
    H_C = np.kron(z,I(3)) + np.kron(np.kron(I(1),z),I(2)) + np.kron(np.kron(I(2),z),I(1)) +  \
          np.kron(I(3),z) - np.kron(np.kron(z,z),I(2)) + np.kron(I(2),np.kron(z,z)) + \
          np.kron(np.kron(np.kron(z,I(1)),z),I(1))
    E = np.conjugate(np.array(B)).T.dot(H_C).dot(np.array(B))[0][0].real # 计算期望
    return (E,gamma,beta)

t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1)
N = 30
beta_lst = np.linspace(0,np.pi,int(N/2))
gamma_lst = np.linspace(0,2*np.pi,N) 
#*********** 并行版本 ******************
table = Parallel(n_jobs=-1)(delayed(QaoaGetPara)(beta,gamma,t) for beta,gamma in itertools.product(beta_lst,gamma_lst)) #并行版本
table = pd.DataFrame(table,columns = ["E","gamma","beta"])
row_idx = table["E"].idxmax()
row = table.loc[row_idx,["gamma", "beta"]] # E_max对应列
pureState = QaoaGetPara(row["gamma"],row["beta"],t,True) # 返回该最优量子线路输出的纯态
State = getBaseProb(pureState) # 返回每个基态的概率